In [33]:
from pyomo.environ import *
import pandas as pd

## Conjuntos 
Del modelo de optimización

| **Término**  | **Descripción**                |
|--------------|--------------------------------|
| $cacopios$   | Conjunto de centros de acopio  |
| $clientes$   | Conjunto de clientes           |
| $productos$  | Conjunto de productos          |

In [34]:
cacopios = 'CAcopios'
clientes = 'Clientes'
productos = 'Productos'

# Configuración de archivos CSV

In [35]:
# Configuración de archivos CSV
csv_base = './csv/'

routes = {
    'cic': 'CapInvCA.csv',
    'cia': 'CostoInvCA.csv',
    'ctac': 'CostoTransCAClie.csv'
}

csv_config = {
    'delimiter': ';',
    'decimal': ','
}

## Centros de acopio

In [36]:
# Cargar datos
cacopios_df = pd.read_csv(csv_base + 'CAcopios.csv')


## Clientes

In [37]:
clientes_df = pd.read_csv(csv_base + 'Clientes.csv')

## Productos

In [38]:
productos_df = pd.read_csv(csv_base + 'Productos.csv')

### Capacidades 
Obteniendo las capacidades de cada centro de acopio

In [39]:
cap_df = pd.read_csv(csv_base + routes['cic'], **csv_config)

### Costos de Inventario
$$
  CInventario_t = \sum_{j \in J} \sum_{p \in P} CostoInvAcopio_{pjt} InvCA_{pjt} \qquad \forall_t \in T
$$

In [40]:
cia_df = pd.read_csv(csv_base + routes['cia'], **csv_config)

### Costos de Transporte
$$
  CTransporte_t = \sum_{k \in K} \sum_{j \in J} \sum_{p \in P} CostoTransAcopioClie_{pjkt} \qquad \forall_t \in T
$$

In [41]:
ctac_df = pd.read_csv(csv_base + routes['ctac'], **csv_config)

### Demandas
Generando demandas para cada cliente, se usarán como entrada para el modelo

In [42]:
demanda = 80
cap_queso = cap_df[productos_df['Producto'] == 'Queso duro']

KeyError: 'Producto'

# Crear modelo

In [18]:
model = ConcreteModel()

Crear el modelo


# Conjuntos

In [19]:
model.CA = Set(initialize=cacopios_df['CentroAcopio'].unique())
model.P = Set(initialize=cacopios_df['Periodo'].unique())

KeyError: 'CentroAcopio'

# Parametros

In [20]:
# Parámetros
model.Demanda = Param(model.P, within=NonNegativeReals, initialize={p: demanda for p in model.P})
model.Stock = Param(model.CA, model.P, within=NonNegativeReals, initialize=dict(((row['CentroAcopio'], row['Periodo']), row['CapInvCA']) for idx, row in cap_df.iterrows()))
model.TiempoTransp = Param(model.CA, model.CA, within=NonNegativeReals, initialize=dict(((row['CentroOrigen'], row['CentroDestino']), row['CostoTransporte']) for idx, row in ctac_df.iterrows()))
model.ProduccionPotencial = Param(model.CA, model.P, within=NonNegativeReals, initialize=dict(((row['CentroAcopio'], row['Periodo']), row['ProduccionPotencial']) for idx, row in cacopios_df.iterrows()))
model.Precio = Param(model.CA, within=NonNegativeReals, initialize=dict(zip(cacopios_df['CentroAcopio'], cacopios_df['Precio'])))
model.TiempoAlistam = Param(model.CA, within=NonNegativeReals, initialize=dict(zip(cacopios_df['CentroAcopio'], cacopios_df['TiempoAlistamiento'])))
model.TiempoMaxDefinido = Param(within=NonNegativeReals, initialize=cacopios_df['TiempoMaxDefinido'].iloc[0])
model.cTransp = Param(model.CA, model.CA, within=NonNegativeReals, initialize=dict(((row['CentroOrigen'], row['CentroDestino']), row['CostoTransporte']) for idx, row in ctac_df.iterrows()))
model.cTiempo = Param(within=NonNegativeReals, initialize=cacopios_df['CostoTiempo'].iloc[0])

AttributeError: 'ConcreteModel' object has no attribute 'P'

# Variables

In [21]:
model.k = Var(model.CA, model.P, within=NonNegativeReals, bounds=lambda model, ca, p: (0, cap_queso[cap_queso['CentroAcopio'] == ca]['CapInvCA'].values[0]))

AttributeError: 'ConcreteModel' object has no attribute 'CA'

In [22]:
Centro de acopio principal (fijar uno de los centros como principal)

SyntaxError: invalid syntax (965617164.py, line 1)

In [23]:
model.CA_principal = Param(within=Any, initialize='CentroPrincipal')

# Restricciones

Restricción de demanda

In [24]:
def demanda_constraint(model, p):
    return sum(model.k[ca, p] for ca in model.CA) == model.Demanda[p]
model.DemandaConstraint = Constraint(model.P, rule=demanda_constraint)

AttributeError: 'ConcreteModel' object has no attribute 'P'

Restricción de inventario y producción potencial

In [25]:
def inventario_constraint(model, ca, p):
    return model.k[ca, p] <= model.Stock[ca, p] + model.ProduccionPotencial[ca, p]
model.InventarioConstraint = Constraint(model.CA, model.P, rule=inventario_constraint)

AttributeError: 'ConcreteModel' object has no attribute 'CA'

Restricción de tiempo de alistamiento

In [26]:
def tiempo_alistam_constraint(model, ca):
    return model.TiempoAlistam[ca] <= model.TiempoMaxDefinido
model.TiempoAlistamConstraint = Constraint(model.CA, rule=tiempo_alistam_constraint)

AttributeError: 'ConcreteModel' object has no attribute 'CA'

Restricción de tiempo de traslado

In [27]:
def tiempo_transp_constraint(model, ca1, ca2):
    return model.TiempoTransp[ca1, ca2] <= model.TiempoMaxDefinido
model.TiempoTranspConstraint = Constraint(model.CA, model.CA, rule=tiempo_transp_constraint)

AttributeError: 'ConcreteModel' object has no attribute 'CA'

## Función objetivo
$$
Min(F) = CInventario + CTransporte
$$

In [28]:
def objetivo(model):
    return sum(model.k[ca, p] * model.Precio[ca] + model.cTransp[ca, model.CA_principal] + 
               (model.TiempoAlistam[ca] + model.TiempoTransp[ca, model.CA_principal]) * model.cTiempo
               for ca in model.CA for p in model.P)
model.Objetivo = Objective(rule=objetivo, sense=minimize

SyntaxError: incomplete input (3173880608.py, line 5)

In [29]:
Resolver el modelo


SyntaxError: invalid syntax (3950955308.py, line 1)

In [30]:
solver = SolverFactory('glpk')
result = solver.solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpi_6urm1_.glpk.raw --wglp /tmp/tmpsssgjju4.glpk.glp --cpxlp
 /tmp/tmpgmjmjfcg.pyomo.lp
Reading problem data from '/tmp/tmpgmjmjfcg.pyomo.lp'...
1 row, 1 column, 1 non-zero
15 lines were read
Writing problem data to '/tmp/tmpsssgjju4.glpk.glp'...
9 lines were written
GLPK Simplex Optimizer 5.0
1 row, 1 column, 1 non-zero
Preprocessing...
~     0: obj =   1.000000000e+00  infeas =  0.000e+00
OPTIMAL SOLUTION FOUND BY LP PREPROCESSOR
Time used:   0.0 secs
Memory used: 0.0 Mb (39693 bytes)
Writing basic solution to '/tmp/tmpi_6urm1_.glpk.raw'...
11 lines were written


In [31]:
Mostrar resultados

SyntaxError: invalid syntax (3617708748.py, line 1)

In [32]:
model.display()

Model unknown

  Variables:
    None

  Objectives:
    None

  Constraints:
    None
